In [18]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
from sklearn.decomposition import PCA
from numpy.linalg import eig
import os
from GeneralizedProcrustes import GeneralizedProcrustes

In [19]:
df = pd.DataFrame([], columns=['Patient','Landmark','X','Y'])
fileCount = 0
for file in os.listdir('Data'):
    data = np.loadtxt('Data/'+file, delimiter=',', skiprows=1)
    tmp = pd.DataFrame(data, columns=['Landmark', 'X', 'Y'])
    tmp['Patient'] = file
    df = pd.concat((df,tmp))
    fileCount +=1
  
df.to_csv('Landmark.csv')

In [20]:
shapeList = []
for patient in np.unique(df.Patient.values):
    shape = df.loc[df.Patient==patient, ['X','Y']].to_numpy()
    shapeList.append(shape)

newShapes = GeneralizedProcrustes(shapeList, maxIter=5)

Distance between mean shape and reference shape dist=2.9004061809556.
Distance between mean shape and reference shape dist=0.8850038288034967.
Distance between mean shape and reference shape dist=0.8850038288034967.
Distance between mean shape and reference shape dist=0.8850038288034967.
Distance between mean shape and reference shape dist=0.8850038288034966.


In [21]:
df = pd.DataFrame([], columns=['Shape', 'Landmark', 'X', 'Y'])
for i, shape in enumerate(newShapes):
    tmp = pd.DataFrame(np.zeros((shape.shape[0],4)), columns=['Shape', 'Landmark', 'X', 'Y'])
    tmp['X'] = shape[:,0]
    tmp['Y'] = shape[:,1]
    tmp.Shape = i+1
    tmp.Landmark = np.arange(1, shape.shape[0]+1)
    df = pd.concat((df, tmp))
df = df.set_index(['Shape', 'Landmark'])
df.to_csv('Landmarks_Aligned.csv')
df

X         Y
Shape Landmark                    
1     1        -0.052745  0.340972
      2        -0.033538  0.275876
      3         0.055416  0.131399
      4         0.133189  0.024022
      5         0.283922 -0.051963
...                  ...       ...
12    14        0.014537  0.174437
      15        0.094401  0.190778
      16        0.149881   0.23532
      17        0.229915  0.212432
      18        0.077364 -0.009849

[216 rows x 2 columns]

In [22]:
df = pd.read_csv('Landmarks_Aligned.csv', index_col=(0))
shapes = df.index.drop_duplicates()
n = shapes.size
data = np.zeros((2*18, n))
for i, shape in enumerate(shapes):
    landmarks = df.loc[shape].values[:,1:].ravel().T
    data[:,i] = landmarks   

In [26]:
pca = PCA(whiten=True)
pca.fit(data)
cov = pca.get_covariance()
eigva, eigve = eig(cov)

(12, 12)